In [62]:
import pandas as pd
import numpy as np
import random

In [124]:
# Possible Semester dates: a_20189, a_20191, a_20195, a_20199, a_20201, a_20205, a_20215, a_20219, a_20221, a_20225, a_20229, a_20231
semester_date = "a_[20231, 20191]"
num_students = 10063


Training instances SMAC -> used to find the best CPLEX config
{'a_20189': 5262,
 'a_20191': 5189,
 'a_20195': 496,
 'a_20199': 5316,
 'a_20201': 5205,
 'a_20205': 1157,
 'a_20215': 771,
"""

"""
Test instances: run best CPLEX -vs- default
 'a_20219': 5620,
 'a_20221': 5510,
 'a_20225': 1111,
 'a_20229': 5916,
 'a_20231': 5861}


"""


- 'a_20189': 5262, a_20191': 5189,
- 'a_20189': 5262, 'a_20199': 5316,
- 'a_20189': 5262, 'a_20201': 5205,
-  a_20191': 5189, 'a_20199': 5316,
-  a_20191': 5189, 'a_20201': 5205,
- 'a_20199': 5316,'a_20201': 5205,
-  a_20191': 5189,, a_2201: 5510

In [64]:
file = "Exam Sched Prog Datasets.xlsx"
file = r"C:\Users\William Hazen\Documents\GitHub\exam-timetabling-smac\Exam Sched Prog Datasets.xlsx"

rooms = pd.read_excel(file, sheet_name="datasets room caps ")
#courses_20221 = pd.read_excel(file, sheet_name = "20221 course size")
#enrolments_20221 = pd.read_excel(file, sheet_name = "20221 anonymized enrolments")
schedule_20221 = pd.read_excel(file, sheet_name="20221 Final Schedule_fromLSM")

hist_anonymized_enrolments = pd.read_excel(
    file, sheet_name="hist anonymized enrolments")

rooms['Room'] = rooms['Room'].astype(str)
rooms['Room'] = rooms['Bd'].str.cat(rooms['Room'], sep=" ")

schedule_20221['Examdate'] = schedule_20221['Examdate'].astype(str)
schedule_20221['Begin'] = schedule_20221['Begin'].astype(str)
examdate_time_list = (schedule_20221['Examdate'].dropna().astype(
    str) + " : " + schedule_20221['Begin']).dropna().tolist()

ETL = np.unique(examdate_time_list)
examdate_time_list = [i for i in ETL if i != "NaT : nan"]

# new processing



In [65]:

ava_rooms = rooms['Room'].values
ava_room_cap = rooms['Room Cap'].values
hist_anonymized_enrolments['ACAD_ACT_CD'] = hist_anonymized_enrolments['ACAD_ACT_CD'].astype(
    str)
hist_anonymized_enrolments['ACAD_ACT_CD'] = hist_anonymized_enrolments['ACAD_ACT_CD'].str.cat(
    hist_anonymized_enrolments['SECTION_CD'])

semeser_list = np.unique(hist_anonymized_enrolments['SESSION_CD']).tolist()



In [121]:
large_semester_list = [[20231, 20191], [20231, 20199], [20231, 20201], [
    20221, 20199], [20221, 20231], [20221, 20201], [20191, 20221]]


In [122]:
anan_sem = {}
name_list = []
for merge_sem in large_semester_list:
    name = "a_{}".format(merge_sem)
    name_list.append(name)
    anan_sem[name] = hist_anonymized_enrolments[(hist_anonymized_enrolments['SESSION_CD'] == merge_sem[0]) | (
        hist_anonymized_enrolments['SESSION_CD'] == merge_sem[1])]


In [95]:
name_list


['a_[20231, 20191]',
 'a_[20189, 20199]',
 'a_[20189, 20201]',
 'a_[20191, 20199]',
 'a_[20191, 20201]',
 'a_[20221, 20201]',
 'a_[20191, 20221]']

In [123]:
for name in name_list:
    print("name: {}, len: {}" .format(name, len(np.unique(anan_sem[name]["HASHED_PERSON_ID"]))))

name: a_[20231, 20191], len: 10063
name: a_[20231, 20199], len: 9097
name: a_[20231, 20201], len: 9005
name: a_[20221, 20199], len: 7878
name: a_[20221, 20231], len: 7044
name: a_[20221, 20201], len: 7782
name: a_[20191, 20221], len: 8859


In [125]:

def processing_fun(semester, num_student):
    name = []
    exam_list = []
    Student_ID_Exams = []
    
    qwe = semester["HASHED_PERSON_ID"].values
    qwer = np.unique(qwe)
    random_sample_names_generator = (qwer[i] for i in range(num_student))

    for i in random_sample_names_generator:
        name.append(i)
        
    Student_ID_List = random.sample(name, len(name))

    for ID in Student_ID_List:
        Student_ID_Exams.append(semester["ACAD_ACT_CD"][semester.eq(ID).any(1)].values)
        
    for exam in Student_ID_Exams:
        exam_list.append(exam.any())
        
    uniq_exams = np.unique(exam_list)
    new_enrol = pd.DataFrame(semester).copy()
    new_enrol["class"] = 1

    temp_df = new_enrol[new_enrol["HASHED_PERSON_ID"].isin(Student_ID_List)]
    final_temp = temp_df[temp_df["ACAD_ACT_CD"].isin(uniq_exams)]

    courses_enrollments_set = pd.pivot_table(data=final_temp, values="class", index="ACAD_ACT_CD", columns=["HASHED_PERSON_ID"], fill_value=0)
    
    return uniq_exams, Student_ID_List, courses_enrollments_set

t_E, t_S, t_C = processing_fun(anan_sem[semester_date], num_students)

sumHe_s = np.sum(t_C.values, axis=1)

#storing    
exam_set = t_E
student_set = t_S
datetime_slot_set = examdate_time_list
room_set = ava_rooms
room_capacity_set = ava_room_cap
courses_enrollments_set = t_C.values

C:\Users\William Hazen\AppData\Local\Temp\ipykernel_268\2392132662.py:16: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  Student_ID_Exams.append(semester["ACAD_ACT_CD"][semester.eq(ID).any(1)].values)


In [126]:
print("# Exams {}, # Students {}, Course Enrollment shape {}, # Rooms {}, # Room capactiy {}".format(len(exam_set), len(student_set), courses_enrollments_set.shape, len(room_set), len(room_capacity_set)))

# Exams 205, # Students 10063, Course Enrollment shape (205, 10063), # Rooms 69, # Room capactiy 69


In [127]:
rooms[rooms["Room Cap"] == 309]

,Bd,Room,Room Cap,Note
26,EX,EX 100,309,NaN


In [128]:
print("Exam set:" '\n', exam_set,'\n\n', 
      "Course Enrollement Set:" '\n',courses_enrollments_set, '\n\n', 
      "Number of students in course:" '\n',sumHe_s, '\n\n', 
)

Exam set:
 ['AER201H1S' 'AER302H1S' 'AER310H1S' 'AER336H1S' 'AER372H1S' 'AER373H1S'
 'AER406H1S' 'AER510H1S' 'APS105H1S' 'APS106H1S' 'APS111H1S' 'APS112H1S'
 'APS113Y1Y' 'APS150H1S' 'APS164H1S' 'APS281H1S' 'APS305H1S' 'APS322H1S'
 'APS324H1S' 'APS343H1S' 'APS360H1S' 'APS420H1S' 'APS432H1S' 'APS440H1S'
 'APS442H1S' 'APS447H1S' 'APS490Y1Y' 'APS502H1S' 'APS511H1S' 'APS530H1S'
 'BME205H1S' 'BME330H1S' 'BME331H1S' 'BME346H1S' 'BME358H1S' 'BME410H1S'
 'BME412H1S' 'BME430H1S' 'BME498Y1Y' 'BME499Y1Y' 'BME530H1S' 'CHE112H1S'
 'CHE113H1S' 'CHE205H1S' 'CHE210H1S' 'CHE213H1S' 'CHE222H1S' 'CHE223H1S'
 'CHE230H1S' 'CHE305H1S' 'CHE311H1S' 'CHE322H1S' 'CHE333H1S' 'CHE354H1S'
 'CHE375H1S' 'CHE403H1S' 'CHE462H1S' 'CHE469H1S' 'CHE471H1S' 'CHE475H1S'
 'CHE488H1S' 'CHE507H1S' 'CHE568H1S' 'CIV100H1S' 'CIV209H1S' 'CIV214H1S'
 'CIV235H1S' 'CIV250H1S' 'CIV300H1S' 'CIV313H1S' 'CIV324H1S' 'CIV332H1S'
 'CIV382Y1Y' 'CIV401H1S' 'CIV440H1S' 'CIV477H1S' 'CIV498H1S' 'CIV516H1S'
 'CIV549H1S' 'CIV576H1S' 'CIV580H1S' 'CM

In [129]:
print("Room Set:" '\n',room_set, '\n\n', 
      "Room Capacity set:" '\n',room_capacity_set, '\n\n')

Room Set:
 ['BA 1130' 'BA 1160' 'BA 1170' 'BA 1180' 'BA 1190' 'BA 2159' 'BA 2165'
 'BA 2175' 'BA 2185' 'BA 2195' 'BN 3' 'BN 2N' 'BN 2N' 'BN 2S' 'BN 2S'
 'BR 200' 'EJ 120' 'EJ 130' 'EJ 209' 'EJ 320' 'EJ 330' 'EJ BN' 'EJ GT'
 'EJ WH' 'EM 119' 'ES 1050' 'EX 100' 'EX 200' 'EX 300' 'EX 310' 'EX 320'
 'GB 303' 'GB 304' 'HA 410' 'MC 102' 'MC 252' 'MC 254' 'MP 102' 'MP 103'
 'MP 202' 'MP 203' 'MS 2158' 'MS 2170' 'MS 2172' 'MS 3153' 'MS 3154'
 'MY 150' 'MY 315' 'MY 330' 'MY 360' 'MY 380' 'PB B150' 'PB B250' 'RW 110'
 'RW 117' 'SF 2202' 'SF 3202' 'SS 2102' 'SS 2117' 'SS 2118' 'ST VLAD'
 'TC 239' 'WB 116' 'WB 119' 'WB 130' 'WB 219' 'WI 1016' 'WI 1017' 'WY 119'] 

 Room Capacity set:
 [ 44  74  36  36  35  35  50  50  50  50 231 384 192 384 192 125  50  50
  50  45  45 100 100  50  45 109 309 335 108 106 120  49  72  50  94  38
  38  56  56  53  53 143  37  37  79  69 235  48  36  48  32  75  60  43
  43  72 110  52  52  52 120  80  82  30  40  30  48  33  80] 




In [130]:
print("Exam Date and time slot:" '\n',examdate_time_list,'\n\n',
      "Student set:" '\n', student_set,'\n\n')

Exam Date and time slot:
 ['2022-04-11 : 09:00:00', '2022-04-11 : 14:00:00', '2022-04-11 : 19:00:00', '2022-04-12 : 09:00:00', '2022-04-12 : 14:00:00', '2022-04-12 : 19:00:00', '2022-04-13 : 09:00:00', '2022-04-13 : 14:00:00', '2022-04-13 : 19:00:00', '2022-04-14 : 09:00:00', '2022-04-14 : 14:00:00', '2022-04-14 : 19:00:00', '2022-04-18 : 09:00:00', '2022-04-18 : 14:00:00', '2022-04-18 : 19:00:00', '2022-04-19 : 09:00:00', '2022-04-19 : 09:30:00', '2022-04-19 : 14:00:00', '2022-04-19 : 18:30:00', '2022-04-19 : 19:00:00', '2022-04-20 : 09:00:00', '2022-04-20 : 09:30:00', '2022-04-20 : 14:00:00', '2022-04-20 : 18:30:00', '2022-04-20 : 19:00:00', '2022-04-21 : 09:00:00', '2022-04-21 : 09:30:00', '2022-04-21 : 14:00:00', '2022-04-21 : 18:30:00', '2022-04-21 : 19:00:00', '2022-04-22 : 09:00:00', '2022-04-22 : 09:30:00', '2022-04-22 : 14:00:00', '2022-04-22 : 18:30:00', '2022-04-22 : 19:00:00', '2022-04-23 : 09:00:00', '2022-04-23 : 09:30:00', '2022-04-23 : 14:00:00', '2022-04-25 : 09:00:00'